In [155]:
from __future__ import print_function    # (at top of module)
import pandas as pd
import numpy as np
from scipy import stats
import time
import csv
import os
import sys
import re
import itertools
from random import shuffle
import pickle
import operator
from IPython.display import display, Image, HTML
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from textwrap import wrap
%matplotlib inline

######################## Learning ########################
# from sklearn_pandas import DataFrameMapper
# from sklearn.linear_model import LogisticRegression
# from sklearn.cluster import KMeans
# from sklearn import preprocessing

# import tensorflow as tf
# depreacted
# from tensorflow.models.rnn import rnn
# from tensorflow.models.rnn import rnn_cell

#################### Viz and plotting ####################

# from matplotlib.colors import LogNorm

# import plotly
# import plotly.plotly as py
# import plotly.graph_objs as go
# from plotly.tools import FigureFactory as FF

# from geopy.geocoders import Nominatim
# from pygeocoder import Geocoder

########################### SQL ##########################

# from sqlalchemy import create_engine
# from sqlalchemy.sql import text
import psycopg2

########################### Text / Web Stuff ##########################

# import urllib2
# from bs4 import BeautifulSoup as bs
# import wikipedia as wiki
# import re
from difflib import SequenceMatcher
import webbrowser
import chardet

########################### Other ##########################

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pprint


##### for reference #####
# import statsmodels.api as sm
# import pylab as pl
# from mpl_toolkits.mplot3d import Axes3D

In [258]:
def spot_search(row):
    
    song = str(row['TITLE'])
#     print (song)
    artist = str(row['ARTIST'])
    result = {}
    result['exact_song_match'] = False
    pulled_new_artist_stuff = False
    
    # search for track using track and artist name
    tracks = sp.search(q=str(str(song) + ' ' + str(artist)), type='track', limit=1)
    # if search with song name and artist name doesn't return anything, search
    #    only with song name
    if len(tracks['tracks']['items']) == 0:
#         print ("Nothing found. Searching with only song titile.")
        tracks = sp.search(q=str(song), type='track', limit=1)
    # search for the artist to grab artist info
    artists = sp.search(q=artist, type='artist', limit=1)
                
    # Song info
    if len(tracks['tracks']['items']) > 0:
        # song name
        song_name = tracks['tracks']['items'][0]['name']        
        result['song_name'] = song_name
        if (song_name).lower() == song.lower():
            result['exact_song_match'] = True
        # song uri
        song_uri = tracks['tracks']['items'][0]['uri']
        result['song_uri'] = song_uri
        # song popularity
        song_pop = tracks['tracks']['items'][0]['popularity']
        result['song_pop'] = song_pop
        # song preview
        preview_url = tracks['tracks']['items'][0]['preview_url']
        result['preview_url'] = preview_url
        # duration
        duration_ms = tracks['tracks']['items'][0]['duration_ms']
        result['duration_ms'] = duration_ms
        # artist name
        artist_name = tracks['tracks']['items'][0]['artists'][0]['name']
        result['artist_name'] = artist_name
        # artist uri
        artist_uri = tracks['tracks']['items'][0]['artists'][0]['uri']
        result['artist_uri'] = artist_uri
        # track explicit
        song_exp = tracks['tracks']['items'][0]['explicit']
        result['explicit'] = song_exp
        try:
            artist_info = sp.artist(artist_uri)
            pulled_new_artist_stuff = True
        except:
            pass
        
        # get specific song features
        features = sp.audio_features([song_uri])[0]
        if features != None:
            result['acousticness'] = features['acousticness']
            result['energy'] = features['energy']
            result['instrumentalness'] = features['instrumentalness']
            result['key'] = features['key']
            result['liveness'] = features['liveness']
            result['loudness'] = features['loudness']
            result['tempo'] = features['tempo']
            result['valence'] = features['valence']
            result['danceability'] = features['danceability']
        else:
            result['acousticness'] = None
            result['energy'] = None
            result['instrumentalness'] = None
            result['key'] = None
            result['liveness'] = None
            result['loudness'] = None
            result['tempo'] = None
            result['valence'] = None
            result['danceability'] = None
        
#         print ("{} --- song search successful!".format(song))
    else:
#         print ("{} --- song search didn't return anything...".format(song))
        result['song_name'] = None
        result['song_uri'] = None
        result['song_pop'] = None
        result['preview_url'] = None
        result['duration_ms'] = None
        result['artist_uri'] = None
        result['acousticness'] = None
        result['energy'] = None
        result['instrumentalness'] = None
        result['key'] = None
        result['liveness'] = None
        result['loudness'] = None
        result['tempo'] = None
        result['valence'] = None
        result['danceability'] = None
        result['explicit'] = None
      
    # check if using info from song if possible, and 
    if pulled_new_artist_stuff:
        # artist image
        if len(artist_info['images']) > 0:
            artist_image = (artist_info['images'][1]['url'])
        else:
            artist_image = None
        result['artist_img'] = artist_image
        # artist genres
        genres = (artist_info["genres"])
        result['genres'] = genres
        # artist popularity
        pop = (artist_info['popularity'])
        result['artist_pop'] = pop
#         print ("{} --- artist info from song pulled successfully!".format(artist))
    
    # use original artist search info if present
    elif len(artists['artists']['items']) > 0:
        # artist image
        if len(artists['artists']['items'][0]['images']) > 0:
            artist_image = (artists['artists']['items'][0]['images'][1]['url'])
        else:
            artist_image = None
        result['artist_img'] = artist_image
        # artist genres
        genres = (artists['artists']['items'][0]["genres"])
        result['genres'] = genres
        # artist popularity
        pop = (artists['artists']['items'][0]['popularity'])
        result['artist_pop'] = pop
#         print ("{} --- artist search successful!".format(artist))
    else:
#         print ("{} --- artist search didn't return anything...".format(artist))
        result['artist_img'] = None
        result['genres'] = None
        result['artist_pop'] = None
        
#     print('\n')
        
    return result


def extract_dicts(df):
    # breaks down search_dict into other columns
    df['artist_img'] = df.search_dict.apply(lambda x: x['artist_img'] if (x != None or x != float('nan')) else None)
    df['artist_pop'] = df.search_dict.apply(lambda x: x['artist_pop'] if (x != None or x != float('nan')) else None)
    df['exact_song_match'] = df.search_dict.apply(lambda x: x['exact_song_match'] if (x != None or x != float('nan')) else None)
    df['genres'] = df.search_dict.apply(lambda x: x['genres'] if (x != None or x != float('nan')) else None)
    df['song_name'] = df.search_dict.apply(lambda x: x['song_name'] if (x != None or x != float('nan')) else None)
    df['song_pop'] = df.search_dict.apply(lambda x: x['song_pop'] if (x != None or x != float('nan')) else None)
    df['song_uri'] = df.search_dict.apply(lambda x: x['song_uri'] if (x != None or x != float('nan')) else None)
    df['artist_uri'] = df.search_dict.apply(lambda x: x['artist_uri'] if (x != None or x != float('nan')) else None)
    df['preview_url'] = df.search_dict.apply(lambda x: x['preview_url'] if (x != None or x != float('nan')) else None)
    df['duration_ms'] = df.search_dict.apply(lambda x: x['duration_ms'] if (x != None or x != float('nan')) else None)
    df['acousticness'] = df.search_dict.apply(lambda x: x['acousticness'] if (x != None or x != float('nan')) else None)
    df['energy'] = df.search_dict.apply(lambda x: x['energy'] if (x != None or x != float('nan')) else None)
    df['instrumentalness'] = df.search_dict.apply(lambda x: x['instrumentalness'] if (x != None or x != float('nan')) else None)
    df['key'] = df.search_dict.apply(lambda x: x['key'] if (x != None or x != float('nan')) else None)
    df['liveness'] = df.search_dict.apply(lambda x: x['liveness'] if (x != None or x != float('nan')) else None)
    df['loudness'] = df.search_dict.apply(lambda x: x['loudness'] if (x != None or x != float('nan')) else None)
    df['tempo'] = df.search_dict.apply(lambda x: x['tempo'] if (x != None or x != float('nan')) else None)
    df['valence'] = df.search_dict.apply(lambda x: x['valence'] if (x != None or x != float('nan')) else None)
    df['danceability'] = df.search_dict.apply(lambda x: x['danceability'] if (x != None or x != float('nan')) else None)
    df['explicit'] = df.search_dict.apply(lambda x: x['explicit'] if (x != None or x != float('nan')) else None)
    return df

def change_sub_genre_to_genre(df, sub_dict):
    artists = df.artist.unique()
    subs = df.subgenre.unique()
    sub_as_key_dict = {}
    # assign the easy ones
    for sub in subs:
        for key in sub_dict.keys():
            if str(sub) in sub_dict[key] or str(sub) in str(key).lower():
                sub_as_key_dict[str(sub)] = str(key)
    print ('Finished the easy ones.')
                
    # split subgenre and search, vote for genre
    for sub in subs:
        if sub not in sub_as_key_dict.keys():
            voting_dict = {}
            for word in sub.split():
                for key in sub_dict.keys():
                    for ind_sub in sub_dict[key]:
                        if word.lower() in ind_sub.lower() or word.lower() in key.lower():
                            if key not in voting_dict.keys():
                                voting_dict[key] = 1
                            else:
                                voting_dict[key] = voting_dict[key] + 1
            # this fn grabs key of largest value
            if len(voting_dict) > 0:
                gen = max(voting_dict.iteritems(), key=operator.itemgetter(1))[0]
                sub_as_key_dict[str(sub)] = str(gen)
    print ('Finished the intense searching.')
                
    
    # if nothing came up, set to none
    for sub in subs:
        if sub not in sub_as_key_dict.keys():
            sub_as_key_dict[str(sub)] = None
    
    # assign genre to each artist based on single subgenre
    df['genre'] = df.subgenre.apply(lambda x: sub_as_key_dict[str(x)])
    
    # create voting dict for each artists genres
    df['master_genre'] = ''
    for artist in artists:
        vote_dict = {}
        subset = df.loc[df.artist == artist, 'genre']
        for genre in list(subset):
            if genre != None:
                if genre not in vote_dict.keys():
                    vote_dict[genre] = 1
                else:
                    vote_dict[genre] += 1
        # This is where THE MAGIC HAPPENS.
        #     ie a final genre is assigned
        if len(vote_dict) > 0:
            gen = max(vote_dict.iteritems(), key=operator.itemgetter(1))[0]
            df.loc[df.artist == artist,'master_genre'] = str(gen)
 
    return df

def classify_keys(row):
    key_dict = {0:'C', 1:'C#', 2:'D', 3:'D#', 4:'E', 5:'F', 6:'F#',
                7:'G', 8:'G#', 9:'A', 10:'A#', 11:'B'}
    try:
        converted = key_dict[int(row['key'])]
    except:
        converted = None
    return converted

In [278]:
# Load data
songs_raw = pd.read_csv("billboard_top_songs.csv", index_col=0)

# Special for billboard data
songs_raw['YEAR'] = songs_raw.Year.apply(lambda x: int(x))

display(songs_raw.head())

# Load spotipy credentials
# ****This will take looking at spotify's api to get
#     working correctly*****
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

# Load subgenre dict
with open('sub_genre_dict.pkl', 'rb') as f:
        sub_genre_dict = pickle.load(f)

,ARTIST,Rank,TITLE,Year,ARTIST_Full,YEAR
0,Mark Ronson,1,Uptown Funk!,2015.0,Mark Ronson feat. Bruno Mars,2015
1,Ed Sheeran,2,Thinking Out Loud,2015.0,Ed Sheeran,2015
2,Wiz Khalifa,3,See You Again,2015.0,Wiz Khalifa feat. Charlie Puth,2015
3,Fetty Wap,4,Trap Queen,2015.0,Fetty Wap,2015
4,Maroon 5,5,Sugar,2015.0,Maroon 5,2015


In [279]:
songs = songs_raw.copy(deep=True)
new_cols = [
 'genres',
 'artist_img',
 'song_uri',
 'song_name',
 'song_pop',
 'artist_pop',
 'exact_song_match',
 'search_dict',
 'artist_uri',
 'preview_url',
 'duration_ms',
 'acousticness',
 'energy',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'tempo',
 'valence',
 'danceability',
 'explicit']
songs = pd.concat([songs,pd.DataFrame(columns=new_cols)])
songs.head()

,ARTIST,ARTIST_Full,Rank,TITLE,YEAR,Year,acousticness,artist_img,artist_pop,artist_uri,...,key,liveness,loudness,preview_url,search_dict,song_name,song_pop,song_uri,tempo,valence
0,Mark Ronson,Mark Ronson feat. Bruno Mars,1.0,Uptown Funk!,2015.0,2015.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ed Sheeran,Ed Sheeran,2.0,Thinking Out Loud,2015.0,2015.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wiz Khalifa,Wiz Khalifa feat. Charlie Puth,3.0,See You Again,2015.0,2015.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fetty Wap,Fetty Wap,4.0,Trap Queen,2015.0,2015.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Maroon 5,Maroon 5,5.0,Sugar,2015.0,2015.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [280]:
# split up dataframe to bypass timeout
print (len(songs))
# songs1 = songs.loc[:499,:]
# songs2 = songs.loc[500:999]
# songs3 = songs.loc[1000:1499,:]
# songs4 = songs.loc[1500:1999,:]
# songs5 = songs.loc[2000:2499,:]
# songs6 = songs.loc[2500:2999,:]
# songs7 = songs.loc[3000:3499,:]
# songs8 = songs.loc[3500:3999,:]
# songs9 = songs.loc[4000:4499,:]
# songs10 = songs.loc[4500:4999,:]
# songs11 = songs.loc[5000:5499,:]
# songs12 = songs.loc[5500:5999,:]
# songs13 = songs.loc[6000:,:]
print (len(songs2))

6581
500


In [285]:
# Search spotify to scrape audio features and other info
t0 = time.time()
# # songs['search_dict'] = songs.apply(spot_search, axis=1)
# songs1['search_dict'] = songs1.apply(spot_search, axis=1)
# print (str("Finished section 1" + "-"*300))
# songs2['search_dict'] = songs2.apply(spot_search, axis=1)
# print (str("Finished section 2" + "-"*300))

# songs1_extr = extract_dicts(songs1)
# songs2_extr = extract_dicts(songs2)

# songs3['search_dict'] = songs3.apply(spot_search, axis=1)
# print (str("Finished section 3" + "-"*300))
# songs3_extr = extract_dicts(songs3)

# songs4['search_dict'] = songs4.apply(spot_search, axis=1)
# print (str("Finished section 4" + "-"*300))
# songs4_extr = extract_dicts(songs4)

# songs5['search_dict'] = songs5.apply(spot_search, axis=1)
# print (str("Finished section 5" + "-"*300))
# songs5_extr = extract_dicts(songs5)

# songs6['search_dict'] = songs6.apply(spot_search, axis=1)
# print (str("Finished section 6" + "-"*300))
# songs6_extr = extract_dicts(songs6)

# songs7['search_dict'] = songs7.apply(spot_search, axis=1)
# print (str("Finished section 7" + "-"*300))
# songs7_extr = extract_dicts(songs7)

# songs8['search_dict'] = songs8.apply(spot_search, axis=1)
# print (str("Finished section 8" + "-"*300))
# songs8_extr = extract_dicts(songs8)

# t0 = time.time()
# songs9['search_dict'] = songs9.apply(spot_search, axis=1)
# print (str("Finished section 9 in {}".format(time.time()-t0) + "-"*300))
# songs9_extr = extract_dicts(songs9)

# songs10['search_dict'] = songs10.apply(spot_search, axis=1)
# print (str("Finished section 10" + "-"*300))
# songs10_extr = extract_dicts(songs10)

# songs11['search_dict'] = songs11.apply(spot_search, axis=1)
# print (str("Finished section 11" + "-"*300))
# songs11_extr = extract_dicts(songs11)

# songs12['search_dict'] = songs12.apply(spot_search, axis=1)
# print (str("Finished section 12" + "-"*300))
# songs12_extr = extract_dicts(songs12)

# songs13['search_dict'] = songs13.apply(spot_search, axis=1)
# print (str("Finished section 13" + "-"*300))
# songs13_extr = extract_dicts(songs13)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

Finished section 11------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [287]:
# recombine datasets
songs_new = songs1.append(songs2, ignore_index=True)

In [288]:
songs_new = songs_new.append(songs3, ignore_index=True)

In [289]:
songs_new = songs_new.append(songs4, ignore_index=True)

In [290]:
songs_new = songs_new.append(songs5, ignore_index=True)

In [291]:
songs_new = songs_new.append(songs6, ignore_index=True)

In [292]:
songs_new = songs_new.append(songs7, ignore_index=True)

In [293]:
songs_new = songs_new.append(songs8, ignore_index=True)

In [294]:
songs_new = songs_new.append(songs9, ignore_index=True)

In [295]:
songs_new = songs_new.append(songs10, ignore_index=True)

In [296]:
songs_new = songs_new.append(songs11, ignore_index=True)

In [297]:
songs_new = songs_new.append(songs12, ignore_index=True)

In [298]:
songs_new = songs_new.append(songs13, ignore_index=True)

In [299]:
# songs = songs_new
songs_copy = songs_new.copy(deep=True)

In [209]:
blank_dic = {'acousticness': None, 
 'genres': [], 'artist_img': None, 'song_uri': None, 'exact_song_match': False, 'explicit': None, 
 'song_name': None, 'liveness': None, 'tempo': None, 'preview_url': None, 'artist_uri': None, 'song_pop': None, 
 'artist_pop': 62, 'instrumentalness': None, 'key': None, 'duration_ms': None, 'loudness': None, 'valence': None, 
 'energy': None, 'danceability': None}

In [301]:
songs_extracted = songs_copy

In [302]:
# create a new df with subgenres and artists
genre_df = pd.DataFrame(columns=['artist', 'subgenre'])
for index, row in songs_extracted.iterrows():
    if row.genres != None:
        for genre in (row.genres):
            new = {'artist': row.ARTIST, 'subgenre': genre}
            genre_df = genre_df.append(new, ignore_index=True)
print ('Finished this part.')
genre_df.drop_duplicates(inplace=True)
genre_df.reset_index(inplace=True, drop=True)
# add columns classifying each subgenre to a
#    master and choose one master for each artist
genre_df = change_sub_genre_to_genre(genre_df, sub_genre_dict)

Finished this part.
Finished the easy ones.
Finished the intense searching.


In [303]:
# construct df with just artists and master genres
just_master_genres = genre_df.loc[:,['artist', 'master_genre']]
just_master_genres.drop_duplicates(inplace=True)
just_master_genres.reset_index(inplace=True, drop=True)

In [305]:
# replace apostrophes in the Title and Artist column
songs_extracted['TITLE'] = songs_extracted.TITLE.apply(lambda x: str(x).replace("\x89\xdb\xaa", "'"))
songs_extracted['ARTIST'] = songs_extracted.ARTIST.apply(lambda x: str(x).replace("\x89\xdb\xaa", "'"))

In [306]:
# identify songs that weren't able to be matched
unfound = songs_extracted.loc[songs_extracted.song_uri.isnull(), :]

In [307]:
# convert key codes to keys
songs_extracted['converted_keys'] = songs_extracted.apply(lambda x: classify_keys(x), axis=1)

In [310]:
cols_to_write = [
 'genres',
 'artist_img',
 'song_uri',
#  'song_name',
 'song_pop',
 'artist_pop',
 'exact_song_match',
 'search_dict',
 'artist_uri',
 'preview_url',
 'duration_ms',
 'acousticness',
 'energy',
 'instrumentalness',
 'key',
 'converted_keys',
 'liveness',
 'loudness',
 'tempo',
 'valence',
 'danceability',
 'ARTIST',
 'THEME',
 'SPOTIFY_URL',
 'TITLE',
 'YEAR',
 'explicit',
 'Rank']

In [311]:
genre_df.to_csv("genres_and_subs.csv")
just_master_genres.to_csv("artists_and_genres.csv")
unfound.to_csv("missing_songs.csv")
songs_extracted.to_csv("top_songs_more_info.csv", columns=cols_to_write)